In [1]:
import os
import requests
import zipfile
import io
import pandas as pd
import chardet  # para detección automática de encoding

In [38]:
# pip install pandas requests openpyxl
# !pip install chardet
# !pip install unidecode

Correr celda de abajo solo la primera vez para crear el archivo, no es necesario una vez creado el csv

In [42]:
# Carpeta destino
os.makedirs("data/axa", exist_ok=True)

# Columnas oficiales
columnas = [
    "SINIESTRO","LATITUD","LONGITUD","CODIGO POSTAL","CALLE","COLONIA",
    "CAUSA SINIESTRO","TIPO VEHICULO","COLOR","MODELO","NIVEL DAÑO VEHICULO",
    "PUNTO DE IMPACTO","AÑO","MES","DÍA NUMERO","DIA","HORA","ESTADO","CIUDAD",
    "LESIONADOS","RELACION LESIONADOS","EDAD LESIONADO","GENERO LESIONADO",
    "NIVEL LESIONADO","HOSPITALIZADO","FALLECIDO","AMBULANCIA","ARBOL",
    "PIEDRA","DORMIDO","GRUA","OBRA CIVIL","PAVIMENTO MOJADO","EXPLOSION LLANTA",
    "VOLCADURA","PERDIDA TOTAL","CONDUCTOR DISTRAIDO","FUGA","ALCOHOL",
    "MOTOCICLETA","BICICLETA","SEGURO","TAXI","ANIMAL"
]

# Rango de años a descargar
anios = range(2018, 2025)

# Lista de DataFrames
dfs = []

for year in anios:
    url = f"https://files.i2ds.org/OpenDataAxaMx/incidentes_viales_{year}_axa.zip"
    print(f"Descargando {year} ...")

    try:
        # Descargar zip
        resp = requests.get(url)
        resp.raise_for_status()
        z = zipfile.ZipFile(io.BytesIO(resp.content))
        
        # Extraer CSV (usualmente solo hay uno)
        csv_name = [f for f in z.namelist() if f.endswith(".csv")][0]
        with z.open(csv_name) as f:
            # Leer el CSV
            if year < 2020:
                df = pd.read_csv(f, encoding="latin1", low_memory=False)
            else:
                # No tiene encabezados → los asignamos manualmente
                df = pd.read_csv(
                    f,
                    header=None,
                    names=columnas,
                    encoding="latin1",
                    low_memory=False
                )
            
            # Reemplazar valores nulos estándar
            df = df.replace({"\\N": pd.NA, " ": pd.NA, "": pd.NA})
            df["AÑO"] = year  # aseguramos el campo de año

            dfs.append(df)

    except Exception as e:
        print(f"⚠️ Error al procesar {year}: {e}")

# Concatenar todo
axa_all = pd.concat(dfs, ignore_index=True)

print(f"✅ Dataset combinado: {len(axa_all)} filas, {len(axa_all.columns)} columnas")
print("Ejemplo de columnas:", axa_all.columns[:10].tolist())

# Guardar limpio
print("💾 Guardado en data/axa/incidentes_viales_2015_2024.csv")


Descargando 2018 ...
Descargando 2019 ...
Descargando 2020 ...
Descargando 2021 ...
Descargando 2022 ...
Descargando 2023 ...
Descargando 2024 ...
✅ Dataset combinado: 927594 filas, 47 columnas
Ejemplo de columnas: ['SINIESTRO', 'LATITUD', 'LONGITUD', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO', 'TIPO VEHICULO', 'COLOR', 'MODELO']
💾 Guardado en data/axa/incidentes_viales_2015_2024.csv


In [49]:
axa_all = pd.read_csv("data/axa/incidentes_viales_2015_2024.csv", encoding="utf-8", low_memory=False)

In [50]:
axa_all.head()

,SINIESTRO,LATITUD,LONGITUD,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO,...,FUGA,ALCOHOL,MOTOCICLETA,BICICLETA,SEGURO,TAXI,ANIMAL,AÑO,NIVEL DAÑO VEHICULO,DÍA NUMERO
0,4379071,16.8529794,-99.9008608,39300,Av. Costera Miguel AlemÃ¡n,Del PanteÃ³n,COLISION Y/O VUELCO,Auto,GRIS,2005,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
1,4378995,16.8521022,-99.7969736,39715,MEX 200,Leonardo Rodriguez Alcaine,COLISION Y/O VUELCO,Auto,AZUL CIELO,2009,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
2,4379228,16.8593584,-99.8437132,39845,Monte Everest MZA 11 LOTE 1,Praderas de Costa Azul,COLISION Y/O VUELCO,Auto,BLANCO,2013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
3,4379138,17.6388618,-101.5504036,40895,Valentina,La Ropa,COLISION Y/O VUELCO,Auto,BLANCO,2011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
4,4379144,0,0,0,AJUSTADOR EN SITIO NOE RAMOS NAVARRETE,Sin dato,COLISION Y/O VUELCO,Auto,ROJO,2012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN


In [51]:
axa_all.shape

(927594, 47)

In [52]:
print(axa_all.columns.tolist())

['SINIESTRO', 'LATITUD', 'LONGITUD', 'CODIGO POSTAL', 'CALLE', 'COLONIA', 'CAUSA SINIESTRO', 'TIPO VEHICULO', 'COLOR', 'MODELO', 'NIVEL DAÃ\x91O VEHICULO', 'PUNTO DE IMPACTO', 'AÃ\x91O', 'MES', 'DÃ\x8dA NUMERO', 'DIA', 'HORA', 'ESTADO', 'CIUDAD', 'LESIONADOS', 'RELACION LESIONADOS', 'EDAD LESIONADO', 'GENERO LESIONADO', 'NIVEL LESIONADO', 'HOSPITALIZADO', 'FALLECIDO', 'AMBULANCIA', 'ARBOL', 'PIEDRA', 'DORMIDO', 'GRUA', 'OBRA CIVIL', 'PAVIMENTO MOJADO', 'EXPLOSION LLANTA', 'VOLCADURA', 'PERDIDA TOTAL', 'CONDUCTOR DISTRAIDO', 'FUGA', 'ALCOHOL', 'MOTOCICLETA', 'BICICLETA', 'SEGURO', 'TAXI', 'ANIMAL', 'AÑO', 'NIVEL DAÑO VEHICULO', 'DÍA NUMERO']


## Revisando datos para saber si es util

In [53]:
axa_all.isna().sum()

SINIESTRO                  734
LATITUD                  12095
LONGITUD                 12133
CODIGO POSTAL            13852
CALLE                     7571
COLONIA                  40614
CAUSA SINIESTRO          61625
TIPO VEHICULO             6519
COLOR                    10093
MODELO                    2727
NIVEL DAÃO VEHICULO    201548
PUNTO DE IMPACTO        242810
AÃO                    115626
MES                          1
DÃA NUMERO             115626
DIA                          1
HORA                         1
ESTADO                      16
CIUDAD                    3383
LESIONADOS                   1
RELACION LESIONADOS     794162
EDAD LESIONADO           78256
GENERO LESIONADO        794027
NIVEL LESIONADO         810446
HOSPITALIZADO                1
FALLECIDO               794134
AMBULANCIA              926562
ARBOL                   926816
PIEDRA                  923245
DORMIDO                 927145
GRUA                    902471
OBRA CIVIL              927592
PAVIMENT

In [54]:
axa_all = axa_all.drop(['AÃO'], axis='columns')

In [55]:
import unidecode

axa_all.columns = (
    axa_all.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    # .str.replace("ñ", "n")
    # .str.replace("ã±", "n")
    # .str.replace("ã\x91", "n")
    .map(unidecode.unidecode)  # elimina acentos y símbolos raros
)

In [56]:
axa_all.isna().sum()

siniestro                 734
latitud                 12095
longitud                12133
codigo_postal           13852
calle                    7571
colonia                 40614
causa_siniestro         61625
tipo_vehiculo            6519
color                   10093
modelo                   2727
nivel_daao_vehiculo    201548
punto_de_impacto       242810
mes                         1
daa_numero             115626
dia                         1
hora                        1
estado                     16
ciudad                   3383
lesionados                  1
relacion_lesionados    794162
edad_lesionado          78256
genero_lesionado       794027
nivel_lesionado        810446
hospitalizado               1
fallecido              794134
ambulancia             926562
arbol                  926816
piedra                 923245
dormido                927145
grua                   902471
obra_civil             927592
pavimento_mojado       927364
explosion_llanta       923196
volcadura 

Viendo la seccion que es de Sonora:

In [64]:
# Normalizar nombres de estados
axa_all["estado"] = (
    axa_all["estado"]
    .astype(str)
    .str.strip()
    .str.lower()
)
axa_sonora = axa_all[axa_all["estado"] == "sonora"].copy()
axa_sonora.shape

(35271, 46)

In [65]:
axa_sonora.head()

,siniestro,latitud,longitud,codigo_postal,calle,colonia,causa_siniestro,tipo_vehiculo,color,modelo,...,fuga,alcohol,motocicleta,bicicleta,seguro,taxi,animal,ano,nivel_dano_vehiculo,dia_numero
6202,4379064,27.4910014,-109.9610979,85148,SEVILLA,BELLAVISTA,COLISION Y/O VUELCO,Auto,GRIS,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
6203,4379393,27.0601117,-109.4606289,85860,Avenida Tamaulipas,DEPORTIVA,COLISION Y/O VUELCO,Auto,GRIS,2012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
6204,4379463,27.5805792,-109.9271832,85019,CARRETERA GUAYMAS-CD. OBREGON,ESPERANZA,COLISION Y/O VUELCO,Auto,ROJO,2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN
6205,4379613,27.4938221,-109.948499,85000,Avenida Vicente Guerrero,DEL VALLE,COLISION Y/O VUELCO,Auto,AZUL,2017,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2018,NaN,NaN
6206,4379736,27.7213894,-110.3823574,85400,Ciudad ObregÃ³n - Guaymas,Guaymas Centro,COLISION Y/O VUELCO,Auto,ROJO,2016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN


In [66]:
axa_sonora.isna().sum()

siniestro                 21
latitud                  752
longitud                 752
codigo_postal            834
calle                    521
colonia                 2116
causa_siniestro         1902
tipo_vehiculo            499
color                    601
modelo                    88
nivel_daao_vehiculo     6721
punto_de_impacto        7789
mes                        0
daa_numero              4233
dia                        0
hora                       0
estado                     0
ciudad                   394
lesionados                 0
relacion_lesionados    28488
edad_lesionado          2664
genero_lesionado       28524
nivel_lesionado        30728
hospitalizado              0
fallecido              28484
ambulancia             35251
arbol                  35248
piedra                 34863
dormido                35251
grua                   33898
obra_civil             35271
pavimento_mojado       35268
explosion_llanta       34915
volcadura              34902
perdida_total 

In [68]:
# Calcular porcentaje de valores vacíos por columna
missing_percent = (
    axa_sonora.isna().mean().sort_values(ascending=False) * 100
).round(2)

# Mostrar como DataFrame ordenado
missing_df = missing_percent.reset_index()
missing_df.columns = ["columna", "porcentaje_vacios"]

print("Porcentaje de valores vacíos en el dataset de Sonora:")
display(missing_df)


Porcentaje de valores vacíos en el dataset de Sonora:


,columna,porcentaje_vacios
0,obra_civil,100.00
1,pavimento_mojado,99.99
2,ambulancia,99.94
3,dormido,99.94
4,arbol,99.93
5,conductor_distraido,99.92
6,perdida_total,99.91
7,bicicleta,99.87
8,motocicleta,99.04
9,explosion_llanta,98.99
